<a href="https://colab.research.google.com/github/Faisal-NSU/cse498R/blob/main/(CountVectorizer)_CSE498R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**A Sentiment Analysis Model of Food Reviews in Bengali Texts Based on Deep Learning techniques.**

Supervisor- **Afn1**



**Members:**

1.   **Mohd. Istiaq Hossain Junaid,1821577642**
2.   Faisal Hossain
3.   Udayan shah Upal
4.   Anjana tameem(Unofficial)





#**Preprocessing**


**Step 01 - Punctuation Removal**
**Step 02 -**

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import random

import string 
import cufflinks as cf
import seaborn as sns
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import tree
import warnings
warnings.filterwarnings("ignore")
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from numpy.random import seed
seed(1)

import tensorflow
tensorflow.random.set_seed(1)

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning:

The twython library has not been installed. Some functionality from the twitter package will not be available.



In [2]:
import pandas as pd
import io
df = pd.read_csv("https://raw.githubusercontent.com/Faisal-NSU/cse498R/main/Food%20Reviews.csv")
df.head()

,Speech,Label
0,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,1
1,অসাধারণ পরিবেশ! খুব সুন্দর মার্জিত এবং আন্তরিক...,1
2,খাবারের মান যথেষ্ট খারাপ। দামের তুলনায় পরিমান ...,0
3,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,1
4,আমি খাইসি খুবই মজার,1


In [3]:
df['Label'].value_counts()

1    520
0    520
Name: Label, dtype: int64

In [4]:
df.duplicated().sum()

0

# Data Pre-Processing



*   Punctuation Removal
*   Tokenization
*   Stopword Removal
*   Stemming
*   Join




In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def remove_punctuation(txt):
    punctuation_list = str(string.punctuation) + "।"
    txt_nopunct = "".join([c for c in txt if c not in punctuation_list])
    return txt_nopunct

In [7]:
def remove_garbage(a):
  a= "".join(i for i in a if i in ["।"] or 2432 <= ord(i) <= 2559 or ord(i)== 32)
  removed=" ".join(a.split())
  return removed

In [8]:
df['Removed Puncutation'] = df['Speech'].apply(lambda x:remove_punctuation(x))
df.head()

,Speech,Label,Removed Puncutation
0,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,1,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...
1,অসাধারণ পরিবেশ! খুব সুন্দর মার্জিত এবং আন্তরিক...,1,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...
2,খাবারের মান যথেষ্ট খারাপ। দামের তুলনায় পরিমান ...,0,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...
3,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,1,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে
4,আমি খাইসি খুবই মজার,1,আমি খাইসি খুবই মজার


In [9]:
df['remove Eng'] = df['Removed Puncutation'].apply(lambda x:remove_garbage(x))
df

,Speech,Label,Removed Puncutation,remove Eng
0,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,1,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটানোর...
1,অসাধারণ পরিবেশ! খুব সুন্দর মার্জিত এবং আন্তরিক...,1,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...
2,খাবারের মান যথেষ্ট খারাপ। দামের তুলনায় পরিমান ...,0,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...
3,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,1,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে
4,আমি খাইসি খুবই মজার,1,আমি খাইসি খুবই মজার,আমি খাইসি খুবই মজার
...,...,...,...,...
1035,দেখেই খেয়ে ফেলতে ইচ্ছা হচ্ছে 😋😋,1,দেখেই খেয়ে ফেলতে ইচ্ছা হচ্ছে 😋😋,দেখেই খেয়ে ফেলতে ইচ্ছা হচ্ছে
1036,সবসময়ই অসাধারণ রান্না ভাই।😍😍🥰,1,সবসময়ই অসাধারণ রান্না ভাই😍😍🥰,সবসময়ই অসাধারণ রান্না ভাই
1037,চিকেন বিরিয়ানিটা অনেক মজা হবে দেখে মনে হচ...,1,চিকেন বিরিয়ানিটা অনেক মজা হবে দেখে মনে হচ...,চিকেন বিরিয়ানিটা অনেক মজা হবে দেখে মনে হচ্ছে এ...
1038,অসম্ভব সুন্দর বিরানি রেসিপি 👌👌,1,অসম্ভব সুন্দর বিরানি রেসিপি 👌👌,অসম্ভব সুন্দর বিরানি রেসিপি


**2.Bangla Tokenization and Bangla Stopword Removing**




In [10]:
pip install bnlp_toolkit

In [11]:
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords

df['Removed Stopped word'] = df['remove Eng'].apply(lambda x: remove_stopwords(x,stopwords))
df

,Speech,Label,Removed Puncutation,remove Eng,Removed Stopped word
0,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,1,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটানোর...,"[ব্যাস্ততম, জীবনের, একটু, মনরম, পরিবেশে, সময়, ..."
1,অসাধারণ পরিবেশ! খুব সুন্দর মার্জিত এবং আন্তরিক...,1,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,"[অসাধারণ, পরিবেশ, সুন্দর, মার্জিত, আন্তরিক]"
2,খাবারের মান যথেষ্ট খারাপ। দামের তুলনায় পরিমান ...,0,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,"[খাবারের, মান, খারাপ, দামের, তুলনায়, পরিমান, ক..."
3,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,1,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,"[ভালো, বহুত, খেয়েছি, বাসা, একটু, দূরে]"
4,আমি খাইসি খুবই মজার,1,আমি খাইসি খুবই মজার,আমি খাইসি খুবই মজার,"[খাইসি, খুবই, মজার]"
...,...,...,...,...,...
1035,দেখেই খেয়ে ফেলতে ইচ্ছা হচ্ছে 😋😋,1,দেখেই খেয়ে ফেলতে ইচ্ছা হচ্ছে 😋😋,দেখেই খেয়ে ফেলতে ইচ্ছা হচ্ছে,"[দেখেই, খেয়ে, ফেলতে, ইচ্ছা]"
1036,সবসময়ই অসাধারণ রান্না ভাই।😍😍🥰,1,সবসময়ই অসাধারণ রান্না ভাই😍😍🥰,সবসময়ই অসাধারণ রান্না ভাই,"[সবসময়ই, অসাধারণ, রান্না, ভাই]"
1037,চিকেন বিরিয়ানিটা অনেক মজা হবে দেখে মনে হচ...,1,চিকেন বিরিয়ানিটা অনেক মজা হবে দেখে মনে হচ...,চিকেন বিরিয়ানিটা অনেক মজা হবে দেখে মনে হচ্ছে এ...,"[চিকেন, বিরিয়ানিটা, মজা, একটু, খেয়ে, দেখি]"
1038,অসম্ভব সুন্দর বিরানি রেসিপি 👌👌,1,অসম্ভব সুন্দর বিরানি রেসিপি 👌👌,অসম্ভব সুন্দর বিরানি রেসিপি,"[অসম্ভব, সুন্দর, বিরানি, রেসিপি]"


**3. Stemming**

In [12]:
pip install bangla-stemmer

In [13]:
from bangla_stemmer.stemmer import stemmer
wordlist = ['কবিরগুলিকে', 'আমাকে', 'নামাবার']
stmr = stemmer.BanglaStemmer()
stm = stmr.stem(wordlist)
print(stm)

applied first rules..
applied third rules..
applied first rules..
applied fourth rules..
['কবির', 'আমা', 'নামা']


In [14]:
from bangla_stemmer.stemmer import stemmer
stmr = stemmer.BanglaStemmer()

In [15]:
df['stemmed'] = df['Removed Stopped word'].apply(lambda x:stmr.stem(x))
df.head()

Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..

,Speech,Label,Removed Puncutation,remove Eng,Removed Stopped word,stemmed
0,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,1,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটানোর...,"[ব্যাস্ততম, জীবনের, একটু, মনরম, পরিবেশে, সময়, ...","[ব্যাস্ততম, জীবন, একটু, মনরম, পরিবেশ, সময়, কাট..."
1,অসাধারণ পরিবেশ! খুব সুন্দর মার্জিত এবং আন্তরিক...,1,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,"[অসাধারণ, পরিবেশ, সুন্দর, মার্জিত, আন্তরিক]","[অসাধারণ, পরিবেশ, সুন্দর, মার্জিত, আন্তরিক]"
2,খাবারের মান যথেষ্ট খারাপ। দামের তুলনায় পরিমান ...,0,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,"[খাবারের, মান, খারাপ, দামের, তুলনায়, পরিমান, ক...","[খাবারের, মান, খারাপ, দাম, তুলনায়, পরিমান, কম,..."
3,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,1,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,"[ভালো, বহুত, খেয়েছি, বাসা, একটু, দূরে]","[ভালো, বহুত, খেয়েছি, বাসা, একটু, দূরে]"
4,আমি খাইসি খুবই মজার,1,আমি খাইসি খুবই মজার,আমি খাইসি খুবই মজার,"[খাইসি, খুবই, মজার]","[খাইসি, খুব, মজ]"


**4.Join**

In [16]:
df['sentence']=df['stemmed'].apply(' '.join)
df.head()

,Speech,Label,Removed Puncutation,remove Eng,Removed Stopped word,stemmed,sentence
0,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,1,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটা...,ব্যাস্ততম জীবনের একটু মনরম পরিবেশে সময় কাটানোর...,"[ব্যাস্ততম, জীবনের, একটু, মনরম, পরিবেশে, সময়, ...","[ব্যাস্ততম, জীবন, একটু, মনরম, পরিবেশ, সময়, কাট...",ব্যাস্ততম জীবন একটু মনরম পরিবেশ সময় কাটানোর ঘু...
1,অসাধারণ পরিবেশ! খুব সুন্দর মার্জিত এবং আন্তরিক...,1,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,অসাধারণ পরিবেশ খুব সুন্দর মার্জিত এবং আন্তরিক ...,"[অসাধারণ, পরিবেশ, সুন্দর, মার্জিত, আন্তরিক]","[অসাধারণ, পরিবেশ, সুন্দর, মার্জিত, আন্তরিক]",অসাধারণ পরিবেশ সুন্দর মার্জিত আন্তরিক
2,খাবারের মান যথেষ্ট খারাপ। দামের তুলনায় পরিমান ...,0,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,খাবারের মান যথেষ্ট খারাপ দামের তুলনায় পরিমান অ...,"[খাবারের, মান, খারাপ, দামের, তুলনায়, পরিমান, ক...","[খাবারের, মান, খারাপ, দাম, তুলনায়, পরিমান, কম,...",খাবারের মান খারাপ দাম তুলনায় পরিমান কম ইফত প্ল...
3,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,1,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,ভালো বহুত খেয়েছি আমার বাসা থেকে একটু দূরে,"[ভালো, বহুত, খেয়েছি, বাসা, একটু, দূরে]","[ভালো, বহুত, খেয়েছি, বাসা, একটু, দূরে]",ভালো বহুত খেয়েছি বাসা একটু দূরে
4,আমি খাইসি খুবই মজার,1,আমি খাইসি খুবই মজার,আমি খাইসি খুবই মজার,"[খাইসি, খুবই, মজার]","[খাইসি, খুব, মজ]",খাইসি খুব মজ


# **(Count Vectors,Count Vectors + TF-IDF, N-Gram Vectors)**

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import classification_report
from keras import layers, models, optimizers
import sklearn.metrics as metrics

In [18]:

# split the dataset into training and testing datasets 

X=df['sentence']
y=df['Label']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#x_train, x_test, y_train, y_test = train_test_split(df['sentence'], df['Label'], test_size=0.3, random_state=42)


# create a count vectorizer object 
cv = CountVectorizer()
count_matrix = cv.fit_transform(X)
print(len(cv.get_feature_names()))

# transform the training and testing data using count vectorizer object
x_train_count =  cv.transform(x_train)
x_test_count =  cv.transform(x_test)
xtrain_count = x_train_count.toarray()
xtest_count=x_test_count.toarray()

tfidf_tr = TfidfTransformer()
x_train_count_tfidf = tfidf_tr.fit_transform(x_train_count)
x_test_count_tfidf = tfidf_tr.transform(x_test_count)
xtrain_count_tfidf = x_train_count_tfidf.toarray()
xtest_count_tfidf=x_test_count_tfidf.toarray()

# ngram level tf-idf 
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(df['sentence'])
x_train_tfidf_ngram =  tv_ngram.transform(x_train)
x_test_tfidf_ngram =  tv_ngram.transform(x_test)
xtrain_tfidf_ngram = x_train_tfidf_ngram.toarray()
xtest_tfidf_ngram=x_test_tfidf_ngram.toarray()

666


In [19]:
def classifierModel(classifier, Xtrain, Ytrain, Xtest):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print("\nAccuracy : ", accuracy_score(predictions, y_test))
    
    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, predictions, target_names=target_names))

**Random Forrest Classifier**

In [20]:
print("\nRandom Forrest Classifier with Count Vectors: ")
classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_count, y_train, x_test_count)

print("\nRandom Forrest Classifier with Count Vectors + TF-IDF: ")
classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nRandom Forrest Classifier with N-Gram Vectors: ")
classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


Random Forrest Classifier with Count Vectors: 

Accuracy :  0.7451923076923077
              precision    recall  f1-score   support

     class 0       0.75      0.74      0.74       104
     class 1       0.74      0.75      0.75       104

    accuracy                           0.75       208
   macro avg       0.75      0.75      0.75       208
weighted avg       0.75      0.75      0.75       208


Random Forrest Classifier with Count Vectors + TF-IDF: 

Accuracy :  0.7355769230769231
              precision    recall  f1-score   support

     class 0       0.74      0.72      0.73       104
     class 1       0.73      0.75      0.74       104

    accuracy                           0.74       208
   macro avg       0.74      0.74      0.74       208
weighted avg       0.74      0.74      0.74       208


Random Forrest Classifier with N-Gram Vectors: 

Accuracy :  0.7307692307692307
              precision    recall  f1-score   support

     class 0       0.73      0.73      0.

**LinearSVM**

In [21]:
print("\nLinearSVM with Count Vectors: ")
classifierModel(SVC(kernel='linear'), x_train_count, y_train, x_test_count)

print("\nLinearSVM with Count Vectors + TF-IDF: ")
classifierModel(SVC(kernel='linear'), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nLinearSVM with N-Gram Vectors: ")
classifierModel(SVC(kernel='linear'), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


LinearSVM with Count Vectors: 

Accuracy :  0.6875
              precision    recall  f1-score   support

     class 0       0.72      0.62      0.66       104
     class 1       0.66      0.76      0.71       104

    accuracy                           0.69       208
   macro avg       0.69      0.69      0.69       208
weighted avg       0.69      0.69      0.69       208


LinearSVM with Count Vectors + TF-IDF: 

Accuracy :  0.7115384615384616
              precision    recall  f1-score   support

     class 0       0.71      0.72      0.71       104
     class 1       0.72      0.70      0.71       104

    accuracy                           0.71       208
   macro avg       0.71      0.71      0.71       208
weighted avg       0.71      0.71      0.71       208


LinearSVM with N-Gram Vectors: 

Accuracy :  0.7307692307692307
              precision    recall  f1-score   support

     class 0       0.73      0.73      0.73       104
     class 1       0.73      0.73      0.73    

**Naive Bayes**

In [22]:
print("\nNaive Bayes with Count Vectors: ")
classifierModel(MultinomialNB(), x_train_count, y_train, x_test_count)

print("\nNaive Bayes with Count Vectors + TF-IDF: ")
classifierModel(MultinomialNB(), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nNaive Bayes with N-Gram Vectors: ")
classifierModel(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


Naive Bayes with Count Vectors: 

Accuracy :  0.6923076923076923
              precision    recall  f1-score   support

     class 0       0.66      0.80      0.72       104
     class 1       0.74      0.59      0.66       104

    accuracy                           0.69       208
   macro avg       0.70      0.69      0.69       208
weighted avg       0.70      0.69      0.69       208


Naive Bayes with Count Vectors + TF-IDF: 

Accuracy :  0.7019230769230769
              precision    recall  f1-score   support

     class 0       0.68      0.78      0.72       104
     class 1       0.74      0.62      0.68       104

    accuracy                           0.70       208
   macro avg       0.71      0.70      0.70       208
weighted avg       0.71      0.70      0.70       208


Naive Bayes with N-Gram Vectors: 

Accuracy :  0.7355769230769231
              precision    recall  f1-score   support

     class 0       0.70      0.83      0.76       104
     class 1       0.79      

**DecisionTreeClassifier**

In [23]:
from sklearn.tree import DecisionTreeClassifier
print("\nDecisionTreeClassifier with Count Vectors: ")
classifierModel(DecisionTreeClassifier(), x_train_count, y_train, x_test_count)

print("\nDecisionTreeClassifier with Count Vectors + TF-IDF: ")
classifierModel(DecisionTreeClassifier(), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nDecisionTreeClassifier with N-Gram Vectors: ")
classifierModel(DecisionTreeClassifier(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


DecisionTreeClassifier with Count Vectors: 

Accuracy :  0.6971153846153846
              precision    recall  f1-score   support

     class 0       0.69      0.72      0.70       104
     class 1       0.71      0.67      0.69       104

    accuracy                           0.70       208
   macro avg       0.70      0.70      0.70       208
weighted avg       0.70      0.70      0.70       208


DecisionTreeClassifier with Count Vectors + TF-IDF: 

Accuracy :  0.6971153846153846
              precision    recall  f1-score   support

     class 0       0.71      0.66      0.69       104
     class 1       0.68      0.73      0.71       104

    accuracy                           0.70       208
   macro avg       0.70      0.70      0.70       208
weighted avg       0.70      0.70      0.70       208


DecisionTreeClassifier with N-Gram Vectors: 

Accuracy :  0.6971153846153846
              precision    recall  f1-score   support

     class 0       0.72      0.65      0.68       

**Logestic Regression**

In [24]:
from sklearn.linear_model import LogisticRegression
print("\nLogisticRegression with Count Vectors: ")
classifierModel(LogisticRegression(), x_train_count, y_train, x_test_count)

print("\nLogisticRegression with Count Vectors + TF-IDF: ")
classifierModel(LogisticRegression(), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nLogisticRegression with N-Gram Vectors: ")
classifierModel(LogisticRegression(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


LogisticRegression with Count Vectors: 

Accuracy :  0.7115384615384616
              precision    recall  f1-score   support

     class 0       0.72      0.69      0.71       104
     class 1       0.70      0.73      0.72       104

    accuracy                           0.71       208
   macro avg       0.71      0.71      0.71       208
weighted avg       0.71      0.71      0.71       208


LogisticRegression with Count Vectors + TF-IDF: 

Accuracy :  0.7211538461538461
              precision    recall  f1-score   support

     class 0       0.72      0.73      0.72       104
     class 1       0.73      0.71      0.72       104

    accuracy                           0.72       208
   macro avg       0.72      0.72      0.72       208
weighted avg       0.72      0.72      0.72       208


LogisticRegression with N-Gram Vectors: 

Accuracy :  0.7259615384615384
              precision    recall  f1-score   support

     class 0       0.72      0.75      0.73       104
     cla

**Multinomial Naive Bayes**

In [25]:

print("\MultinomialNB with Count Vectors: ")
classifierModel(MultinomialNB(), x_train_count, y_train, x_test_count)

print("\MultinomialNB with Count Vectors + TF-IDF: ")
classifierModel(MultinomialNB(), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\MultinomialNB with N-Gram Vectors: ")
classifierModel(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

\MultinomialNB with Count Vectors: 

Accuracy :  0.6923076923076923
              precision    recall  f1-score   support

     class 0       0.66      0.80      0.72       104
     class 1       0.74      0.59      0.66       104

    accuracy                           0.69       208
   macro avg       0.70      0.69      0.69       208
weighted avg       0.70      0.69      0.69       208

\MultinomialNB with Count Vectors + TF-IDF: 

Accuracy :  0.7019230769230769
              precision    recall  f1-score   support

     class 0       0.68      0.78      0.72       104
     class 1       0.74      0.62      0.68       104

    accuracy                           0.70       208
   macro avg       0.71      0.70      0.70       208
weighted avg       0.71      0.70      0.70       208

\MultinomialNB with N-Gram Vectors: 

Accuracy :  0.7355769230769231
              precision    recall  f1-score   support

     class 0       0.70      0.83      0.76       104
     class 1       0.79